<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

### 1.1 - Basic Config

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "../data/papers"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### 1.2 - Inspect Input Data

We have a bnunch of datasets in [data](../data) folder.  Examine them

We will use one of them or feel free to bring your own data.

### 1.3 - Set input/output path variables for the pipeline

In [2]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### 1.4 -  Import Common python modules

In [3]:
import os
import sys


from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 2.1 -  Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='../data/papers' --> output='output/01_parquet_out'


### 2.2 - Execute 

In [5]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")


21:54:37 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
21:54:37 INFO - pipeline id pipeline_id
21:54:37 INFO - code location None
21:54:37 INFO - data factory data_ is using local data access: input_folder - ../data/papers output_folder - output/01_parquet_out
21:54:37 INFO - data factory data_ max_files -1, n_sample -1
21:54:37 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
21:54:37 INFO - orchestrator pdf2parquet started at 2024-09-18 21:54:37
21:54:37 INFO - Number of files is 2, source profile {'max_file_size': 5.398899078369141, 'min_file_size': 2.112621307373047, 'total_file_size': 7.5115203857421875}
21:54:37 INFO - Initializing models


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

21:55:11 INFO - Completed 1 files (50.0%) in 0.499 min
21:55:28 INFO - Completed 2 files (100.0%) in 0.782 min
21:55:28 INFO - Done processing 2 files, waiting for flush() completion.
21:55:28 INFO - done flushing in 0.0 sec
21:55:28 INFO - Completed execution in 0.841 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 1min 52s, sys: 1.43 s, total: 1min 53s
Wall time: 53.2 s


### 2.3 -  Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention-is-all-you-need.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf
1,Granite_Foundation_Models.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 3.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 3.2 - Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_python import DocChunkPythonTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(DocChunkPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

21:55:29 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
21:55:29 INFO - pipeline id pipeline_id
21:55:29 INFO - code location None
21:55:29 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
21:55:29 INFO - data factory data_ max_files -1, n_sample -1
21:55:29 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
21:55:29 INFO - orchestrator doc_chunk started at 2024-09-18 21:55:29
21:55:29 INFO - Number of files is 2, source profile {'max_file_size': 0.0879373550415039, 'min_file_size': 0.03559398651123047, 'total_file_size': 0.12353134155273438}
21:55:29 INFO - Comp

✅ Stage:2 completed successfully
CPU times: user 965 ms, sys: 114 ms, total: 1.08 s
Wall time: 1.09 s


### 3.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 266
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (266, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
158,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,A. Foundation Model Evaluation Framework\n2) I...,$.main-text[158],7,"[48.24153519, 648.87231445, 300.22531128, 707...."
259,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,JAPANESE FINANCE BENCHMARK EVALUATION RESULTS\...,$.tables[9],18,"[66.48503876, 285.29275513, 545.31109619, 335...."
19,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,3 Model Architecture\nThe Transformer follows ...,$.main-text[32],3,"[106.86346436, 323.58956909, 505.24719238, 354..."


## Step-4:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 4.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_docid_out'


### 4.2 - Execute 

In [11]:
%%time 

from doc_id_transform_python import DocIDPythonTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = PythonTransformLauncher(DocIDPythonTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

21:55:29 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
21:55:29 INFO - pipeline id pipeline_id
21:55:29 INFO - code location None
21:55:29 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_docid_out
21:55:29 INFO - data factory data_ max_files -1, n_sample -1
21:55:29 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
21:55:29 INFO - orchestrator doc_id started at 2024-09-18 21:55:29
21:55:29 INFO - Number of files is 2, source profile {'max_file_size': 0.04591846466064453, 'min_file_size': 0.024618148803710938, 'total_file_size': 0.07053661346435547}
21:55:29 INFO - Completed 1 files (50.0%) in 0.0 min
21:55:29 INFO - Completed 2 files (100.0%) in 0.0 min
21:55:29 INFO - Done processing 2 files, waiting for flush() completion.
21:55

✅ Stage:3 completed successfully
CPU times: user 15 ms, sys: 2.02 ms, total: 17 ms
Wall time: 13.8 ms


### 4.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 15)
Output data dimensions (rows x columns)=  (266, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id
79,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,"7 Conclusion\nIn this work, we presented the T...",$.main-text[131],10,"[107.62519836, 371.36807251, 504.08255005, 402...",5c2480ff983f40a73dfc6da56793ee498042ec8a000a4f...,257
82,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,7 Conclusion\nThe code we used to train and ev...,$.main-text[134],10,"[107.08765411, 245.77929688, 505.05099487, 266...",d5f76800c895c554018a4bfe8751e6313993661fe0cf53...,260
88,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,Granite Foundation Models\nAbstract -We introd...,$.main-text[2],1,"[47.95682526, 581.7409668, 300.33987427, 650.7...",6b38b1bb18656824905a09dddf5c2be704196a006daaaf...,0


## Step-5: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 5.1 - Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_docid_out' --> output='output/04_exact_dedupe_out'


### 5.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_python import EdedupPythonTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # ededup parameters
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(EdedupPythonTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

21:55:29 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None}
21:55:29 INFO - pipeline id pipeline_id
21:55:29 INFO - code location None
21:55:29 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/04_exact_dedupe_out
21:55:29 INFO - data factory data_ max_files -1, n_sample -1
21:55:29 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
21:55:29 INFO - orchestrator ededup started at 2024-09-18 21:55:29
21:55:29 INFO - Number of files is 2, source profile {'max_file_size': 0.053391456604003906, 'min_file_size': 0.02887439727783203, 'total_file_size': 0.08226585388183594}
21:55:29 INFO - Starting from the beginning
21:55:29 INFO - Completed 1 files (50.0%) in 0.0 min
21:55:29 INFO - Completed 2 files (100.0%) in 0.0 min
21:55:29 INFO - 

✅ Stage:4 completed successfully
CPU times: user 24.5 ms, sys: 2.05 ms, total: 26.5 ms
Wall time: 22.8 ms


### 5.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 17)
Output data dimensions (rows x columns)=  (266, 18)
Input chunks before exact dedupe : 266
Output chunks after exact dedupe : 266
Duplicate chunks removed :   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id,removed
5,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,Attention Is All You Need\nAidan N. Gomez ∗ † ...,$.main-text[8],1,"[234.82455444, 475.27728271, 339.99435425, 508...",3bdf2978bed016a6996cf37ffdd1fb96248cac96fccf70...,183,[]
87,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,Attention Visualizations Input-Input Layer5\nF...,$.main-text[190],15,"[107.43354034, 157.36341858, 504.06988525, 189...",67626adb815bf2b27871df24d538ddc10ae68a3fbbd238...,265,[]
194,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,B. Granite Model Evaluation and Comparison\nmp...,$.tables[3],10,"[87.39694214, 407.41442871, 261.10140991, 477....",3b19639d275e79607b82eb2ddd7b7a4b0db0c20760fd85...,106,[]


## Step-6: Fuzzy Dedup

**Fuzzy dedupe is currently available in RAY version**

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 6.1 - Set Input/output Folder

In [16]:
STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_exact_dedupe_out' --> output='output/05_fuzzy_dedupe_out'


### 6.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher


# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

21:55:29 INFO - Running locally
21:55:29 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
21:55:29 INFO - data factory data_ is using local data access: input_folder - output/04_exact_dedupe_out output_folder - output/05_fuzzy_dedupe_out
21:55:29 INFO - data factory data_ max_files -1, n_sample -1
21:55:29 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
21:55:29 INFO - pipeline id pipeline_id
21:55:29 INFO - code location None
21:55:29 INFO - number o

✅ Stage:5 completed successfully
CPU times: user 445 ms, sys: 340 ms, total: 785 ms
Wall time: 27.2 s


### 6.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 18)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash
212,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,"B. Downstream Documentation\nInput, Phase = . ...",$.tables[5],12,"[99.78625488, 67.24668884, 512.03955078, 687.1...",124,[],-1
10,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,1 Introduction\nRecurrent models typically fac...,$.main-text[19],2,"[107.33866882, 546.35638428, 504.39059448, 632...",188,[],-1
89,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,Granite Foundation Models\nIndex Terms -founda...,$.main-text[3],1,"[48.05226898, 531.65960693, 300.17034912, 570....",1,[],-1


## Step-7:   Text encoding

Encode text for the vector storage.

### 7.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 7.2 - Execute

In [20]:
%%time 

from text_encoder_transform_python import TextEncoderPythonTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = PythonTransformLauncher(TextEncoderPythonTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

21:55:56 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
21:55:56 INFO - pipeline id pipeline_id
21:55:56 INFO - code location None
21:55:56 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
21:55:56 INFO - data factory data_ max_files -1, n_sample -1
21:55:56 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
21:55:56 INFO - orchestrator text_encoder started at 2024-09-18 21:55:56
21:55:56 INFO - Number of files is 2, source profile {'max_file_size': 0.04749584197998047, 'min_file_size': 0.02600860595703125, 'total_file_size': 0.07350444793701172}
21:56:00 INFO - Completed 1 files (50.0%) in 0.016 min
21:56:00 INFO - Completed 2 files (100.0%) in 0.024 min
21:56:00 

✅ Stage:6 completed successfully
CPU times: user 1.89 s, sys: 161 ms, total: 2.05 s
Wall time: 4.09 s


### 7.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 19)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,embeddings
229,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,"A. Model Outputs from Safety Tasks\nOutput : ""...",$.main-text[380],16,"[68.22934723, 327.2671814, 300.39501953, 372.7...",141,[],-1,"[0.02780273, 0.0015434993, -0.01713702, 0.0060..."
44,attention-is-all-you-need.pdf,15,4,193,19f6c470-36a0-4451-addc-788ac4e7ddb8,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T21:55:28.021651,16.976467,attention-is-all-you-need.pdf,4 Why Self-Attention\nIn this section we compa...,$.main-text[80],6,"[106.83399963, 217.19508362, 505.65368652, 270...",222,[],-1,"[-0.03718457, -0.08758642, 0.048269093, -0.042..."
260,Granite_Foundation_Models.pdf,20,13,444,b93fc398-a714-4a78-a4b0-b4a01b203330,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-18T21:55:11.033250,29.943425,Granite_Foundation_Models.pdf,JAPANESE FINANCE BENCHMARK EVALUATION RESULTS\...,$.tables[10],18,"[94.07535553, 50.58311462, 517.87585449, 236.5...",172,[],-1,"[0.028512888, -0.034300514, -0.02429659, 0.032..."


## Step-8: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
